In [1]:
import os
import re

import numpy as np

from numpy import array
from keras.models import Sequential , Model
from keras.layers import Dense ,Input 
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import merge

Using TensorFlow backend.


### TimeDistributed and return_sequence

https://datascience.stackexchange.com/questions/10836/the-difference-between-dense-and-timedistributeddense-of-keras

In [2]:
# prepare sequence
length = 5
seq = array([i/float(length) for i in range(length*4)])
X = seq.reshape(-1, length, 4)
y = seq.reshape(-1, length, 4)
# define LSTM configuration
n_neurons = 3#length

x = Input(shape = (length,4))
out = LSTM(n_neurons,  return_sequences=False,return_state=False)(x)
#out = TimeDistributed(Dense(1))(out)
model = Model(inputs=x , outputs=out)
model.summary()

model.predict(X)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5, 4)              0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 3)                 96        
Total params: 96
Trainable params: 96
Non-trainable params: 0
_________________________________________________________________


array([[ 0.38044807,  0.85386306,  0.37628984]], dtype=float32)

In [3]:
# prepare sequence
length = 5
seq = array([i/float(length) for i in range(length*4)])
X = seq.reshape(-1, length, 4)
y = seq.reshape(-1, length, 4)
# define LSTM configuration
n_neurons = 3#length

x = Input(shape = (length,4))
out = LSTM(n_neurons,  return_sequences=True,return_state=False)(x)
#out = TimeDistributed(Dense(1))(out)
model = Model(inputs=x , outputs=out)
model.summary()

model.predict(X)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 5, 4)              0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 3)              96        
Total params: 96
Trainable params: 96
Non-trainable params: 0
_________________________________________________________________


array([[[ 0.06427185, -0.0725867 ,  0.02028074],
        [ 0.12223752, -0.14583044,  0.02891432],
        [ 0.11952052, -0.14092228,  0.        ],
        [ 0.08312379, -0.09177356,  0.        ],
        [ 0.04013606, -0.03976355,  0.        ]]], dtype=float32)

In [4]:
# prepare sequence
length = 5
seq = array([i/float(length) for i in range(length*4)])
X = seq.reshape(-1, length, 4)
y = seq.reshape(-1, length, 4)
# define LSTM configuration
n_neurons = 3#length

x = Input(shape = (length,4))
out = LSTM(n_neurons,  return_sequences=True,return_state=False)(x)
out = TimeDistributed(Dense(1))(out)
model = Model(inputs=x , outputs=out)
model.summary()

model.predict(X)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 5, 4)              0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 5, 3)              96        
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 1)              4         
Total params: 100
Trainable params: 100
Non-trainable params: 0
_________________________________________________________________


array([[[ 0.09903315],
        [ 0.21050203],
        [ 0.28685269],
        [ 0.36754459],
        [ 0.46470472]]], dtype=float32)

In [5]:
from keras.layers import Lambda, Input
from keras.models import Model
#from keras.backend import softmax #無法設定axis
from keras.activations import softmax #可設定axis
import numpy as np

inputo = Input(shape=(5,1))
double = Lambda(lambda x: softmax(x , axis=1))(inputo)

model = Model(inputs=[inputo], outputs=[double])
model.compile(optimizer='sgd', loss='mse')
model.summary()
data = np.arange(5).reshape((-1,5,1))
print(model.predict(data))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 5, 1)              0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 5, 1)              0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
[[[ 0.01165623]
  [ 0.03168492]
  [ 0.08612854]
  [ 0.23412167]
  [ 0.63640863]]]


In [6]:
data

array([[[0],
        [1],
        [2],
        [3],
        [4]]])

In [7]:
x = data.reshape(-1)    

np.exp(x)/sum(np.exp(x))

array([ 0.01165623,  0.03168492,  0.08612854,  0.23412166,  0.63640865])

## Attention Layer

In [8]:
from keras.layers import Multiply
from keras.layers import Lambda, Input
from keras.models import Model
#from keras.backend import softmax #無法設定axis
from keras.activations import softmax #可設定axis
import numpy as np

length=2
my_input = Input(shape=(length,5))
ls = LSTM(10,return_sequences=True)(my_input)

td = TimeDistributed(Dense(1,activation='linear',use_bias=False))(ls)
sm = Lambda(lambda x : softmax(x,axis=1) , name='softmax')(td)

attn = Multiply()([sm,ls])

out = LSTM(5,return_sequences=True)(attn)
model = Model(inputs=[my_input] , outputs=[out])
model.compile(loss = 'mse',optimizer='adam')
model.summary()

model_1 = Model(inputs=[my_input] , outputs=[ls])
model_2 = Model(inputs=[my_input] , outputs=[sm])
model_3 = Model(inputs=[my_input] , outputs=[attn])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 2, 5)          0                                            
____________________________________________________________________________________________________
lstm_4 (LSTM)                    (None, 2, 10)         640         input_5[0][0]                    
____________________________________________________________________________________________________
time_distributed_2 (TimeDistribu (None, 2, 1)          10          lstm_4[0][0]                     
____________________________________________________________________________________________________
softmax (Lambda)                 (None, 2, 1)          0           time_distributed_2[0][0]         
___________________________________________________________________________________________

In [9]:
x = np.arange(10).reshape((-1,length,5))
y = x.reshape((-1))[::-1].reshape((-1,length,5))
#model.fit(x,y,epochs=1000,verbose=0)
model.predict(x,batch_size=1)

array([[[ 0.00945939, -0.00881535,  0.01615339, -0.00747715,  0.00801516],
        [-0.00384565, -0.06710139,  0.05737033, -0.00500772,  0.03147374]]], dtype=float32)

In [10]:
a1 = model_1.predict(x)
a2 = model_2.predict(x)
a3 = model_3.predict(x)

a1.shape,a2.shape,a3.shape

((1, 2, 10), (1, 2, 1), (1, 2, 10))

In [11]:
(a1[0][1]*a2[0][1]==a3[0][1]).all()

True

In [12]:
120*0.802*0.5

48.120000000000005